In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.15)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.68 sec - Loss 0.214564 - ACC 73.32% - ACC Mean 73.32% - AUC 78.08% - AUC Mean 78.08% - Deter 000
Ite 00050 - 0.94 sec - Loss 0.101095 - ACC 73.82% - ACC Mean 74.52% - AUC 78.67% - AUC Mean 80.26% - Deter 022
Ite 00078 - 0.92 sec - Loss 0.084206 - ACC 71.76% - ACC Mean 74.10% - AUC 79.88% - AUC Mean 79.71% - Deter 050
Early stop ite 78, rollback to correction of ite 28, whith acc of 75.84% and auc of 81.85%
OUR METHOD RUN 1/10 - acc: 75.84% +- 0.0% - auc: 81.85% +- 0.0%
Ite 00000 - 0.90 sec - Loss 0.223886 - ACC 73.23% - ACC Mean 73.23% - AUC 79.53% - AUC Mean 79.53% - Deter 000
Ite 00050 - 0.88 sec - Loss 0.115086 - ACC 73.51% - ACC Mean 73.67% - AUC 79.93% - AUC Mean 79.14% - Deter 039
Ite 00100 - 0.92 sec - Loss 0.086057 - ACC 73.60% - ACC Mean 73.47% - AUC 80.02% - AUC Mean 80.11% - Deter 001
Ite 00150 - 0.85 sec - Loss 0.072395 - ACC 74.55% - ACC Mean 74.33% - AUC 79.40% - AUC Mean 80.64% - Deter 033
Ite 00167 - 0.81 sec - Loss 0.068865 - ACC 75.00% - ACC Mean 74.45% 

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 71.41% +- 0.0% - auc: 78.98% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 55.31% +- 3.63% - auc: 59.36% +- 2.79%
SFIL RUN 2/10 - acc: 53.88% +- 3.41% - auc: 58.72% +- 2.28%
SFIL RUN 3/10 - acc: 56.44% +- 0.84% - auc: 59.74% +- 2.62%
SFIL RUN 4/10 - acc: 56.4% +- 1.98% - auc: 58.76% +- 2.83%
SFIL RUN 5/10 - acc: 57.86% +- 1.95% - auc: 61.5% +- 4.46%
SFIL RUN 6/10 - acc: 56.27% +- 2.77% - auc: 60.13% +- 2.22%
SFIL RUN 7/10 - acc: 54.33% +- 2.5% - auc: 57.8% +- 1.73%
SFIL RUN 8/10 - acc: 57.5% +- 6.14% - auc: 61.56% +- 3.95%
SFIL RUN 9/10 - acc: 56.79% +- 4.74% - auc: 60.71% +- 6.47%
SFIL RUN 10/10 - acc: 55.28% +- 3.16% - auc: 58.54% +- 2.3%
SFIL GLOBAL - acc: 56.01% +- 1.23% - auc: 59.68% +- 1.22%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 71.51% +- 3.84% - auc: 78.53% +- 2.38%
PFIL RUN 2/10 - acc: 70.83% +- 2.5% - auc: 75.53% +- 3.6%
PFIL RUN 3/10 - acc: 72.32% +- 3.9% - auc: 77.86% +- 5.26%
PFIL RUN 4/10 - acc: 72.31% +- 1.89% - auc: 77.29% +- 2.49%
PFIL RUN 5/10 - acc: 71.25% +- 4.34% - auc: 76.46% +- 3.84%
PFIL RUN 6/10 - acc: 71.64% +- 2.89% - auc: 76.93% +- 3.65%
PFIL RUN 7/10 - acc: 70.61% +- 1.89% - auc: 76.46% +- 1.67%
PFIL RUN 8/10 - acc: 72.47% +- 2.17% - auc: 77.28% +- 3.02%
PFIL RUN 9/10 - acc: 72.11% +- 4.6% - auc: 77.9% +- 5.35%
PFIL RUN 10/10 - acc: 72.02% +- 4.28% - auc: 75.07% +- 3.32%
PFIL GLOBAL - acc: 71.71% +- 0.62% - auc: 76.93% +- 1.03%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 62.22% +- 5.89% - auc: 67.22% +- 5.73%
SPOL RUN 2/10 - acc: 60.74% +- 6.27% - auc: 65.17% +- 5.01%
SPOL RUN 3/10 - acc: 58.28% +- 3.42% - auc: 62.53% +- 5.17%
SPOL RUN 4/10 - acc: 60.15% +- 2.08% - auc: 62.77% +- 3.61%
SPOL RUN 5/10 - acc: 58.44% +- 3.21% - auc: 61.85% +- 3.62%
SPOL RUN 6/10 - acc: 56.01% +- 2.24% - auc: 60.61% +- 3.25%
SPOL RUN 7/10 - acc: 55.31% +- 1.46% - auc: 57.74% +- 1.14%
SPOL RUN 8/10 - acc: 56.06% +- 2.9% - auc: 61.57% +- 2.44%
SPOL RUN 9/10 - acc: 60.46% +- 4.25% - auc: 65.56% +- 5.93%
SPOL RUN 10/10 - acc: 57.52% +- 5.5% - auc: 60.71% +- 4.93%
SPOL GLOBAL - acc: 58.52% +- 2.21% - auc: 62.57% +- 2.64%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 71.81% +- 2.4% - auc: 76.75% +- 2.13%
PPOL RUN 2/10 - acc: 70.25% +- 3.29% - auc: 76.26% +- 4.52%
PPOL RUN 3/10 - acc: 70.08% +- 4.02% - auc: 75.68% +- 5.23%
PPOL RUN 4/10 - acc: 71.66% +- 2.53% - auc: 77.88% +- 1.0%
PPOL RUN 5/10 - acc: 68.42% +- 3.64% - auc: 76.04% +- 4.2%
PPOL RUN 6/10 - acc: 72.21% +- 3.38% - auc: 77.3% +- 5.2%
PPOL RUN 7/10 - acc: 70.96% +- 2.64% - auc: 76.44% +- 2.63%
PPOL RUN 8/10 - acc: 70.77% +- 1.66% - auc: 76.58% +- 2.3%
PPOL RUN 9/10 - acc: 70.57% +- 3.24% - auc: 76.57% +- 3.67%
PPOL RUN 10/10 - acc: 71.04% +- 3.55% - auc: 78.34% +- 5.94%
PPOL GLOBAL - acc: 70.78% +- 1.02% - auc: 76.78% +- 0.78%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
